In [2]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np

# MLP 모델 정의
import sys
sys.path.append('/content/')
from utils import MLP, MLP_Tanh, MLP_Sigmoid, DataLoader_exper

# 하이퍼파라미터 설정
batch_size = 128        # 배치 크기
test_batch_size = 1000  # 테스트 배치 크기 (메모리 효율을 위해 크게 설정)
learning_rate = 1e-3    # 학습률 (0.001)
nb_epochs = 3           # 에포크 수

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 로딩 및 전처리
train_loader, test_loader = DataLoader_exper(batch_size, test_batch_size)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mnist/train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

mnist/test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [3]:
# 반복되는 실험 코드를 함수로 묶기 (4.2 부분 활용)

def run_experiment(experiment_name, model_class, learning_rate, nb_epochs=3):
    print(f"\n=== {experiment_name} ===")

    # model, criterion, optimizer 준비
    model = model_class().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 훈련 루프
    for epoch in range(nb_epochs):
        model.train()
        for batch in train_loader:
            imgs, labels = batch["image"].to(device), batch["label"].to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # 평가 루프
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for batch in test_loader:
            imgs, labels = batch["image"].to(device), batch["label"].to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    # 최종 테스트 정확도
    final_accuracy = 100 * correct_test / total_test
    print(f"최종 테스트 정확도: {final_accuracy:.2f}%")
    return final_accuracy

In [4]:
# 실험 1-1 : [하이퍼파라미터 튜닝] 학습률 증

exper1_2 = [0.002, 0.004, 0.006, 0.008, 0.01, 0.1]

result_exper1_2 = { }

print("실험 1-1: 학습률 증가")

for lr in exper1_2:
      acc = run_experiment(
        experiment_name=f"학습률 {lr:.3f}",
        model_class=MLP,
        learning_rate=lr,
        nb_epochs=3
    )
      result_exper1_2[f"{lr}"] = acc

실험 1-1: 학습률 증가

=== 학습률 0.002 ===
최종 테스트 정확도: 97.08%

=== 학습률 0.004 ===
최종 테스트 정확도: 97.22%

=== 학습률 0.006 ===
최종 테스트 정확도: 96.50%

=== 학습률 0.008 ===
최종 테스트 정확도: 96.65%

=== 학습률 0.010 ===
최종 테스트 정확도: 95.82%

=== 학습률 0.100 ===
최종 테스트 정확도: 45.92%


In [5]:
# 실험 1-2 : [하이퍼파라미터 튜닝] 학습률 감소

exper1_2 = [0.0009, 0.0007, 0.0005, 0.0003, 0.0001, 0.00001]

result_exper1_2 = { }

print("실험 1-2: 학습률 감소")

for lr in exper1_2:
      acc = run_experiment(
        experiment_name=f"학습률 {lr:.5f}",
        model_class=MLP,
        learning_rate=lr,
        nb_epochs=3
    )
      result_exper1_2[f"{lr}"] = acc

실험 1-2: 학습률 감소

=== 학습률 0.00090 ===
최종 테스트 정확도: 96.96%

=== 학습률 0.00070 ===
최종 테스트 정확도: 96.73%

=== 학습률 0.00050 ===
최종 테스트 정확도: 96.22%

=== 학습률 0.00030 ===
최종 테스트 정확도: 95.18%

=== 학습률 0.00010 ===
최종 테스트 정확도: 92.82%

=== 학습률 0.00001 ===
최종 테스트 정확도: 85.01%


In [6]:
# 실험 2 : [모델 구조 개선] 다른 활성화 함수 사용

print("실험 2: 다른 활성화 함수 사용")

exper2 = [
    ("Tanh 활성화 함수", MLP_Tanh),
    ("Sigmoid 활성화 함수", MLP_Sigmoid)
]

result_exper2 = { }

for name, model_class in exper2:
  acc = run_experiment(
        experiment_name=f"{name} 사용",
        model_class=model_class,
        learning_rate=1e-3,
        nb_epochs=3
  )

  result_exper2[name] = acc

실험 2: 다른 활성화 함수 사용

=== Tanh 활성화 함수 사용 ===
최종 테스트 정확도: 96.44%

=== Sigmoid 활성화 함수 사용 ===
최종 테스트 정확도: 95.21%
